In [1]:
import os
import zipfile
from pathlib import Path

import requests
import pandas as pd

# Data Retrieval

In [48]:
UCI_ZIP_URL = "https://archive.ics.uci.edu/static/public/502/online+retail+ii.zip"
DATA_DIR = Path("data/raw")
ZIP_PATH = DATA_DIR / "online_retail_ii.zip"
XLSX_NAME = "online_retail_II.xlsx"
XLSX_PATH = DATA_DIR / XLSX_NAME

In [49]:
def download_file(url: str, out_path: Path, chunk_size: int = 1024 * 1024) -> None:
    out_path.parent.mkdir(parents=True, exist_ok=True)
    if out_path.exists() and out_path.stat().st_size > 0:
        print(f"[skip] {out_path} already exists ({out_path.stat().st_size} bytes)")
        return

    print(f"[download] {url}")
    with requests.get(url, stream=True, timeout=60) as r:
        r.raise_for_status()
        total = int(r.headers.get("Content-Length", 0))

        tmp_path = out_path.with_suffix(out_path.suffix + ".part")
        downloaded = 0
        with open(tmp_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=chunk_size):
                if chunk:
                    f.write(chunk)
                    downloaded += len(chunk)
                    if total:
                        pct = 100 * downloaded / total
                        print(f"\r  {downloaded:,}/{total:,} bytes ({pct:5.1f}%)", end="")
        if total:
            print()
        tmp_path.replace(out_path)
    print(f"[saved] {out_path}")


def unzip(zip_path: Path, extract_dir: Path) -> None:
    extract_dir.mkdir(parents=True, exist_ok=True)
    
    if (extract_dir / XLSX_NAME).exists():
        print(f"[skip] already extracted: {(extract_dir / XLSX_NAME)}")
        return

    print(f"[unzip] {zip_path} -> {extract_dir}")
    with zipfile.ZipFile(zip_path, "r") as zf:
        zf.extractall(extract_dir)
    print("[done] extraction complete")


def load_excel(xlsx_path: Path) -> dict[str, pd.DataFrame]:
    # The Excel has two sheets: "Year 2009-2010" and "Year 2010-2011"
    print(f"[read] {xlsx_path}")
    sheets = pd.read_excel(xlsx_path, sheet_name=None, engine="openpyxl")
    print("[sheets]", list(sheets.keys()))
    for k, df in sheets.items():
        print(f"  - {k}: {df.shape[0]:,} rows x {df.shape[1]} cols")
    return sheets


In [51]:
download_file(UCI_ZIP_URL, ZIP_PATH)
unzip(ZIP_PATH, DATA_DIR)

if not XLSX_PATH.exists():
    # Some zips may extract into a nested folder; search for the xlsx if needed
    candidates = list(DATA_DIR.rglob("*.xlsx"))
    raise FileNotFoundError(f"Could not find {XLSX_NAME}. Found: {[str(c) for c in candidates]}")

sheets = load_excel(XLSX_PATH)

[download] https://archive.ics.uci.edu/static/public/502/online+retail+ii.zip
[saved] data/raw/online_retail_ii.zip
[unzip] data/raw/online_retail_ii.zip -> data/raw
[done] extraction complete
[read] data/raw/online_retail_II.xlsx
[sheets] ['Year 2009-2010', 'Year 2010-2011']
  - Year 2009-2010: 525,461 rows x 8 cols
  - Year 2010-2011: 541,910 rows x 8 cols


# EDA

In [52]:
# Data Overview
df_all = pd.concat(sheets.values(), ignore_index=True)
column = df_all
print("First three rows: \n", df_all.head(3))
print("All records number: ", df_all.shape)
print("Column name: ", df_all.columns.tolist())

First three rows: 
   Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   

          InvoiceDate  Price  Customer ID         Country  
0 2009-12-01 07:45:00   6.95      13085.0  United Kingdom  
1 2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
2 2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
All records number:  (1067371, 8)
Column name:  ['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'Country']


In [53]:
# Null value exploration
print("Null counts per column: \n", df_all.isna().sum().sort_values(ascending=False))

# Uniqueness
n_products = df_all["StockCode"].nunique(dropna=True)
n_customers = df_all["Customer ID"].nunique(dropna=True)
print(f"\n Unique products: {n_products:,}")
print(f"Unique customers: {n_customers:,}")

Null counts per column: 
 Customer ID    243007
Description      4382
Invoice             0
StockCode           0
Quantity            0
InvoiceDate         0
Price               0
Country             0
dtype: int64

 Unique products: 5,305
Unique customers: 5,942


In [54]:
# Country-level exploration
n_countries = df_all["Country"].nunique(dropna=True)
uk_count = (df_all["Country"] == "United Kingdom").sum()
uk_pct = uk_count / len(df_all) * 100
print(f"Unique countries: {n_countries:,}")
print(f"United Kingdom rows: {uk_count:,} ({uk_pct:.2f}%)")

# Range of InvoiceDate
print("\n InvoiceDate range:")
invoice_dt = pd.to_datetime(df_all["InvoiceDate"], errors="coerce")
print(f"Min InvoiceDate: {invoice_dt.min()}")
print(f"Max InvoiceDate: {invoice_dt.max()}")
print(f"Invalid/NaT InvoiceDate: {invoice_dt.isna().sum():,}")

Unique countries: 43
United Kingdom rows: 981,330 (91.94%)

 InvoiceDate range:
Min InvoiceDate: 2009-12-01 07:45:00
Max InvoiceDate: 2011-12-09 12:50:00
Invalid/NaT InvoiceDate: 0


In [55]:
# Duplication Check
dup_mask_all = df_all.duplicated(keep=False)
print("Exact-duplicate rows (all columns):", dup_mask_all.sum())

if dup_mask_all.any():
    dup_all = df_all[dup_mask_all].copy()

    grp_sizes = dup_all.groupby(list(df_all.columns), dropna=False).size().sort_values(ascending=False)
    print("\nTop duplicate groups (all columns):")
    print(grp_sizes.head(10))

    first_key = grp_sizes.index[0]

    key_df = pd.DataFrame([first_key], columns=df_all.columns)

    first_group = dup_all.merge(key_df, on=list(df_all.columns), how="inner")
    print("\nExample of an exact-duplicate group:")
    print(first_group.to_string(index=False))

Exact-duplicate rows (all columns): 67242

Top duplicate groups (all columns):
Invoice  StockCode  Description                          Quantity  InvoiceDate          Price  Customer ID  Country       
555524   22698      PINK REGENCY TEACUP AND SAUCER       1         2011-06-05 11:37:00  2.95   16923.0      United Kingdom    20
         22697      GREEN REGENCY TEACUP AND SAUCER      1         2011-06-05 11:37:00  2.95   16923.0      United Kingdom    12
537224   70007      HI TEC ALPINE HAND WARMER            1         2010-12-05 16:24:00  1.65   13174.0      United Kingdom    10
572861   22775      PURPLE DRAWERKNOB ACRYLIC EDWARDIAN  12        2011-10-26 12:46:00  1.25   14102.0      United Kingdom     8
536749   21415      CLAM SHELL SMALL                     2         2010-12-02 13:49:00  2.10   17976.0      United Kingdom     6
536874   22866      HAND WARMER SCOTTY DOG DESIGN        1         2010-12-03 11:35:00  2.10   16891.0      United Kingdom     6
537042   21579      LOLI

In [56]:
# Duplication Check
dup_all = df_all[dup_mask_all].copy()

if len(dup_all) > 0:
    # 2) For each duplicated invoice, count how many unique InvoiceDate values it has
    #    If the duplicates are “split/recording artifacts”, many invoices will have exactly 1 unique timestamp.
    inv_dt_nunique = (
        dup_all.groupby("Invoice")["InvoiceDate"]
        .nunique(dropna=False)
        .sort_values(ascending=False)
    )

    print("\nAmong duplicated rows, unique InvoiceDate counts per Invoice (top 20):")
    print(inv_dt_nunique.head(20))

    share_same_ts = (inv_dt_nunique == 1).mean() * 100
    print(f"\n% of duplicated invoices where all duplicated lines share the exact same InvoiceDate: {share_same_ts:.2f}%")

    # 3) Show an example invoice whose duplicated lines all share the same InvoiceDate
    example_invoice = inv_dt_nunique[inv_dt_nunique == 1].index[0]
    ex = dup_all[dup_all["Invoice"] == example_invoice].sort_values(
        ["InvoiceDate", "StockCode", "Description", "Quantity", "Price", "Customer ID", "Country"],
        kind="mergesort"
    )
else:
    print("No exact-duplicate rows found.")


Among duplicated rows, unique InvoiceDate counts per Invoice (top 20):
Invoice
519730    2
536591    2
567183    2
543812    1
544068    1
544040    1
543973    1
543907    1
543899    1
543816    1
543815    1
543814    1
489517    1
544074    1
543804    1
543803    1
543802    1
543800    1
543731    1
543653    1
Name: InvoiceDate, dtype: int64

% of duplicated invoices where all duplicated lines share the exact same InvoiceDate: 99.94%
